# Binomial Pricing Model

In [ ]:
import numpy as np 
from math import exp, sqrt

In [ ]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [ ]:

class BinomialPriceModel(object):
    def __init__(self,BPM_Paramaters):
        self.So, self.h = BPM_Paramaters["So"], BPM_Paramaters["h"]
        self.k, self.sig = BPM_Paramaters["k"], BPM_Paramaters["sig"]
        self.r, self.delta = BPM_Paramaters["r"], BPM_Paramaters["delta"]
        
    def solve(self,option,flavor):      
        # Gather u and d values at each step
        u_d = []
        [u_d.append([i,self.u_value(h),self.d_value(h),h]) for i,h in\
                 enumerate(self.h[::-1])]
        
        # Gather values at each step
        s_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
        s_values[0,0] = self.So
        for i in range(1,len(u_d)+1):
            s_values[0,i] = u_d[i-1][1]*s_values[0,i-1]
            for j in range(1,i+1):
                s_values[j,i] = u_d[i-1][2]*s_values[j-1,i-1]     
                
        # Get European Values
        vo_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
        
        for i in range(len(u_d)+1):
            if option == "Call":
                vo_values[i,len(u_d)] = max(0, s_values[i,len(u_d)]-self.k)
            elif option == "Put":  
                vo_values[i,len(u_d)] = max(0,self.k-s_values[i,len(u_d)])
                       
        v_values = vo_values   
        for t in list(reversed(range(len(u_d)))):
            for i in range(t+1):
                a = self.Pu_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i,t+1]
                b = self.Pd_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i+1,t+1]
                v_values[i,t] = a+b
                
        # Get American values 
        if flavor == "A": 
            Av_values = vo_values
            Av_bool_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
            
            for t in list(reversed(range(len(u_d)))):
                for i in list(range(t+1)):
                    if option == "Put":
                        e = self.k - s_values[i,t]
                    if option == "Call":
                        e = s_values[i,t] - self.k
                        
                    a = self.Pu_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i,t+1]
                    b = self.Pd_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i+1,t+1]
                    x = a+b
                    Av_values[i,t] = max(e,x)
                    if e > x: 
                        Av_bool_values[i,t] = 1
                        
            mask = Av_values[:,-1] != 0  
            Av_bool_values[:,-1] = mask
        
        if flavor == "E":
            asset_value = v_values[0,0] 
            return asset_value, s_values
        
        if flavor == "A": 
            return (Av_values,Av_bool_values,s_values)
            
    def u_value(self,h):
        return exp((self.r-self.delta)*h + self.sig*sqrt(h))

    def d_value(self,h):
        return exp((self.r-self.delta)*h - self.sig*sqrt(h))
    
    def Pu_value(self,u,d,h):
        return exp(-self.r*h)*((exp((self.r-self.delta)*h)-d)/(u-d))
     
    def Pd_value(self,u,d,h):
        return exp(-self.r*h)*((u-exp((self.r-self.delta)*h))/(u-d))    
    

In [ ]:
"""
Examples
10 nodes and experiation is one year; n=9 and h=1/9
11 nodes and experation is 10 years; n=10 and h=1

"""
n = 9 # number of nodes, not including intial, also 
h = 1/9 # lentgh of period in years 

BPM_Paramaters = {
"So": 100,
"h": [h]*n, 
"k": 80, 
"sig": .4,
"r": 0.01,
"delta": 0,      
}

model = BinomialPriceModel(BPM_Paramaters)

In [ ]:
EC_price, s_values = model.solve(option="Call",flavor="E")
print("European Call Price: ", np.round(EC_price,8))

In [ ]:
EP_price, s_values = model.solve(option="Put",flavor="E")
print("European Put Price: ",np.round(EP_price,8))

In [ ]:
AP_price, AP_bool,s_values = model.solve(option="Put",flavor="A")
np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print("American Put Price: ","\n",np.round(AP_price,8))

In [ ]:
AC_price, AC_bool,s_values = model.solve(option="Call",flavor="A")
print(AC_price.shape)
print("American Call Price: ")
matprint(np.round(AC_price,5))
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})